# categorization test

In [ ]:
import os
import numpy as np
import joblib
import time
import timeit
import joblib
import pickle
from ExpressionExpert_Functions import init_Exp2, Data_Src_Load, make_DataDir, split_train_test, ExpressionScaler, Sequence_Conserved_Adjusted, MyRF, MySV, MyGB
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
Name_Dict = init_Exp2('config_Pput.txt')

ML_Date = Name_Dict['ML_Date']
File_Base = Name_Dict['Data_File'].split('.')[0]
Data_Folder = 'data-{}'.format(File_Base) 
Measure_Numb = int(Name_Dict['Library_Expression'])
ML_Regressor = Name_Dict['ML_Regressor'][:-1]
ML_Type = Name_Dict['ML_Regressor'][-1]
Y_Col_Name = eval(Name_Dict['Y_Col_Name'])
Meas_Name = '{}_ML'.format(Y_Col_Name[0])


In [ ]:
SeqDat = Data_Src_Load(Name_Dict)
SeqDat.head(3)

In [ ]:
print('new train-test')
train_size = 1 - eval(Name_Dict['TestRatio'])
# split number '1' because we only use one final test set. Cross validation comes later
gss = GroupShuffleSplit(n_splits=1, train_size=train_size)
X = SeqDat['Sequence']
y = SeqDat[Y_Col_Name]
groups = SeqDat['Sequence_letter-encrypted'].str.upper()
Train_Idx, Test_Idx = list(gss.split(X, y, groups))[0]
SeqTest = SeqDat.iloc[Test_Idx].reset_index(drop=True)
SeqTrain = SeqDat.iloc[Train_Idx].reset_index(drop=True)

In [ ]:
test_ratio = .1
split_number = 100

# removing non-informative positions where no base diversity exists, base one hot encoding
SeqTrain_Hadj, Positions_removed, PSEntropy = Sequence_Conserved_Adjusted(SeqTrain, Name_Dict, Entropy_cutoff=float(Name_Dict['Entropy_cutoff']))
SeqOH = SeqTrain_Hadj
test_ratio = .1
split_number = 100
Num=100
Validation_cutoff=.1
AddFeat = eval(Name_Dict['Add_Feat'])

Sequence_Samples, Sequence_Positions, Sequence_Bases = np.array(SeqOH['OneHot'].values.tolist()).shape
X = np.array(SeqOH['OneHot'].values.tolist()).reshape(Sequence_Samples,Sequence_Positions*Sequence_Bases)
X = np.append(X,np.array(SeqOH[AddFeat]), axis=1)
Y = SeqOH[Meas_Name].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit, GridSearchCV
import numpy as np


groups = SeqOH['Sequence_letter-encrypted']
Number_Estimators = np.arange(20,40,2)
Max_Features = np.arange(10,30,2)
min_samples_split = np.arange(3,4,1)
max_depth = np.arange(3,5)
min_samples_leaf = np.array([2])
param_grid = [{'bootstrap':[False], 'n_estimators': Number_Estimators, 'max_features': Max_Features, 'min_samples_split': min_samples_split, 'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}]
# Group shuffle split removes groups with identical sequences from the development set
# This is more realistic for parameter estimation
cv = GroupShuffleSplit(n_splits=Num, test_size=Validation_cutoff, random_state=42)

forest_grid = RandomForestClassifier()

grid_forest = GridSearchCV(forest_grid, param_grid, cv=cv, n_jobs=-1)
grid_forest.fit(X, Y, groups)
ML_Best = grid_forest.best_estimator_


In [ ]:
MyReg = MyRF(SeqTrain_Hadj, test_ratio, split_number, Meas_Name, eval(Name_Dict['Response_Value']), AddFeat)
ML_Best = MyReg.best_estimator_
print('Finished')

In [ ]:
Measure_Name = '{}_ML'.format(Y_Col_Name[0])

Regressor_File = os.path.join(Data_Folder, '{}_{}_{}_{}-Regressor.pkl'.format(ML_Date, File_Base, Measure_Name.replace(' ','-'), Name_Dict['ML_Regressor']))
Parameter_File = os.path.join(Data_Folder, '{}_{}_{}_{}-Params.pkl'.format(ML_Date, File_Base, Measure_Name.replace(' ','-'), Name_Dict['ML_Regressor']))

ML_DictName = '{}_Regressor'.format(Y_Col_Name)
ML_Best = joblib.load(Regressor_File)
# I assume the parameters have been generated in the same run as the regressor itself and is located in the same directory following the default naming scheme
Data_Prep_Params = pickle.load(open(Parameter_File,'rb'))
# extracting the positions that were removed because of insufficient information content
Positions_removed = Data_Prep_Params['Positions_removed']
# if the data was standardized we load the corresponding function


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, GroupShuffleSplit
from ExpressionExpert_Functions import list_onehot
from sklearn.metrics import balanced_accuracy_score, classification_report, roc_auc_score, f1_score
from sklearn.metrics import make_scorer 
my_score = make_scorer(f1_score, average='weighted')

CVsplit = 25
X_tmp = list_onehot(list(np.delete(np.array(list(SeqTrain['Sequence_label-encrypted'])),Positions_removed, axis=1)))
X_train = np.array(X_tmp).reshape(len(SeqTrain.index),-1)
AddFeat = eval(Name_Dict['Add_Feat'])
# adding the additional feature, here GC-content
X_train = np.append(X_train,SeqTrain[AddFeat].values, axis=1)
# activity prediction of training set with best random forest estimator
Y_train_pred = ML_Best.predict(X_train)

# Test set prediction
# removing sequence positions that were missing in the feature vector for ml
# getting one-hot encodings from the original train and test data
X_tmp = list_onehot(list(np.delete(np.array(list(SeqTest['Sequence_label-encrypted'])),Positions_removed, axis=1)))
X_test = np.array(X_tmp).reshape(len(SeqTest.index),-1)
# adding the additional feature, here GC-content
X_test = np.append(X_test,SeqTest[AddFeat].values, axis=1)
# activity prediction of training set with best random forest estimator
Y_test_pred = ML_Best.predict(X_test)

# corresponding observations scaled
#     Scaler_DictName = '{}_Scaler'.format(Y_Col_Name[Meas_Idx])
Y_train = SeqTrain[Meas_Name].values
Y_test = SeqTest[Meas_Name].values

# r2_train[Meas_Idx] = r2_score(Y_train[:, Meas_Idx], Y_train_pred[:, Meas_Idx])
# r2_test[Meas_Idx] = r2_score(Y_test[:, Meas_Idx], Y_test_pred[:, Meas_Idx])
# rmse_train[Meas_Idx] = sqrt(mean_squared_error(Y_train[:, Meas_Idx], Y_train_pred[:, Meas_Idx]))
# rmse_test[Meas_Idx] = sqrt(mean_squared_error(Y_test[:, Meas_Idx], Y_test_pred[:, Meas_Idx]))

# cross-validation scoring
cv = GroupShuffleSplit(n_splits=CVsplit, test_size=.1, random_state=42)
# if applicable correcting target variable according to the standardization
# Y_train2 = Y_train
groups = SeqTrain['Sequence_letter-encrypted']
scores = cross_val_score(ML_Best, X_train, Y_train, groups=groups, scoring=my_score, cv=cv)# , groups=groups, scoring=my_r2_score

print('Cross validation for ML-Type: {}'.format(ML_Type))
print('R2 Statistic: {:.2f} (+/-{:.2f})'.format(scores.mean(), scores.std()*2))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
classes = ['low','medium','high']
plot_confusion_matrix(ML_Best, X_test, Y_test, display_labels=classes)
plot_confusion_matrix(ML_Best, X_train, Y_train, display_labels=classes)
plt.show()

In [ ]:

print('Balanced accuracy: {}'.format(balanced_accuracy_score(Y_test, Y_test_pred)))
print('Weighted f1 score: {:.2f}'.format(f1_score(Y_test, Y_test_pred, average='weighted')))
print(classification_report(Y_test, Y_test_pred))